# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Aug 1 2022 8:30AM,243100,10,0085916848,ISDIN Corporation,Released
1,Aug 1 2022 8:30AM,243100,10,0085916876,ISDIN Corporation,Released
2,Aug 1 2022 8:30AM,243100,10,0085916877,ISDIN Corporation,Released
3,Aug 1 2022 8:30AM,243100,10,0085916878,ISDIN Corporation,Released
4,Aug 1 2022 8:30AM,243100,10,0085916885,ISDIN Corporation,Released
5,Aug 1 2022 8:30AM,243100,10,0085916928,ISDIN Corporation,Released
6,Aug 1 2022 8:30AM,243100,10,0085916930,ISDIN Corporation,Released
7,Jul 29 2022 3:27PM,243099,10,SOTAH0000742,"Nextsource Biotechnology, LLC",Released
8,Jul 29 2022 3:20PM,243098,10,CTF0011344,"Citieffe, Inc.",Released
9,Jul 29 2022 2:39PM,243095,10,MTR-22-0241,"Senseonics, Incorporated",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
13,243094,Released,1
14,243095,Released,2
15,243098,Released,1
16,243099,Released,1
17,243100,Released,7


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
243094,NaN,NaN,1.0
243095,NaN,NaN,2.0
243098,NaN,NaN,1.0
243099,NaN,NaN,1.0
243100,NaN,NaN,7.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
243024,0.0,1.0,0.0
243054,8.0,7.0,3.0
243071,0.0,1.0,0.0
243076,0.0,0.0,30.0
243077,0.0,0.0,10.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
243024,0,1,0
243054,8,7,3
243071,0,1,0
243076,0,0,30
243077,0,0,10


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,243024,0,1,0
1,243054,8,7,3
2,243071,0,1,0
3,243076,0,0,30
4,243077,0,0,10


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,243024,,1,
1,243054,8,7,3
2,243071,,1,
3,243076,,,30
4,243077,,,10


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Aug 1 2022 8:30AM,243100,10,ISDIN Corporation
7,Jul 29 2022 3:27PM,243099,10,"Nextsource Biotechnology, LLC"
8,Jul 29 2022 3:20PM,243098,10,"Citieffe, Inc."
9,Jul 29 2022 2:39PM,243095,10,"Senseonics, Incorporated"
11,Jul 29 2022 2:35PM,243094,10,"Citieffe, Inc."
12,Jul 29 2022 2:14PM,243088,10,"ClearSpec, LLC"
14,Jul 29 2022 1:52PM,243083,10,ISDIN Corporation
40,Jul 29 2022 1:52PM,243086,20,Alumier Labs Inc.
59,Jul 29 2022 1:32PM,243080,10,"ACI Healthcare USA, Inc."
71,Jul 29 2022 1:23PM,243079,16,Sartorius Stedim North America


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Aug 1 2022 8:30AM,243100,10,ISDIN Corporation,,,7
1,Jul 29 2022 3:27PM,243099,10,"Nextsource Biotechnology, LLC",,,1
2,Jul 29 2022 3:20PM,243098,10,"Citieffe, Inc.",,,1
3,Jul 29 2022 2:39PM,243095,10,"Senseonics, Incorporated",,,2
4,Jul 29 2022 2:35PM,243094,10,"Citieffe, Inc.",,,1
5,Jul 29 2022 2:14PM,243088,10,"ClearSpec, LLC",,,2
6,Jul 29 2022 1:52PM,243083,10,ISDIN Corporation,,10,16
7,Jul 29 2022 1:52PM,243086,20,Alumier Labs Inc.,,,19
8,Jul 29 2022 1:32PM,243080,10,"ACI Healthcare USA, Inc.",,,12
9,Jul 29 2022 1:23PM,243079,16,Sartorius Stedim North America,,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 1 2022 8:30AM,243100,10,ISDIN Corporation,7,,
1,Jul 29 2022 3:27PM,243099,10,"Nextsource Biotechnology, LLC",1,,
2,Jul 29 2022 3:20PM,243098,10,"Citieffe, Inc.",1,,
3,Jul 29 2022 2:39PM,243095,10,"Senseonics, Incorporated",2,,
4,Jul 29 2022 2:35PM,243094,10,"Citieffe, Inc.",1,,
5,Jul 29 2022 2:14PM,243088,10,"ClearSpec, LLC",2,,
6,Jul 29 2022 1:52PM,243083,10,ISDIN Corporation,16,10,
7,Jul 29 2022 1:52PM,243086,20,Alumier Labs Inc.,19,,
8,Jul 29 2022 1:32PM,243080,10,"ACI Healthcare USA, Inc.",12,,
9,Jul 29 2022 1:23PM,243079,16,Sartorius Stedim North America,1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 1 2022 8:30AM,243100,10,ISDIN Corporation,7,,
1,Jul 29 2022 3:27PM,243099,10,"Nextsource Biotechnology, LLC",1,,
2,Jul 29 2022 3:20PM,243098,10,"Citieffe, Inc.",1,,
3,Jul 29 2022 2:39PM,243095,10,"Senseonics, Incorporated",2,,
4,Jul 29 2022 2:35PM,243094,10,"Citieffe, Inc.",1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 1 2022 8:30AM,243100,10,ISDIN Corporation,7.0,NaN,NaN
1,Jul 29 2022 3:27PM,243099,10,"Nextsource Biotechnology, LLC",1.0,NaN,NaN
2,Jul 29 2022 3:20PM,243098,10,"Citieffe, Inc.",1.0,NaN,NaN
3,Jul 29 2022 2:39PM,243095,10,"Senseonics, Incorporated",2.0,NaN,NaN
4,Jul 29 2022 2:35PM,243094,10,"Citieffe, Inc.",1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 1 2022 8:30AM,243100,10,ISDIN Corporation,7.0,0.0,0.0
1,Jul 29 2022 3:27PM,243099,10,"Nextsource Biotechnology, LLC",1.0,0.0,0.0
2,Jul 29 2022 3:20PM,243098,10,"Citieffe, Inc.",1.0,0.0,0.0
3,Jul 29 2022 2:39PM,243095,10,"Senseonics, Incorporated",2.0,0.0,0.0
4,Jul 29 2022 2:35PM,243094,10,"Citieffe, Inc.",1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2673944,85.0,17.0,8.0
12,243024,0.0,1.0,0.0
16,486150,1.0,1.0,0.0
20,243086,19.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2673944,85.0,17.0,8.0
1,12,243024,0.0,1.0,0.0
2,16,486150,1.0,1.0,0.0
3,20,243086,19.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,85.0,17.0,8.0
1,12,0.0,1.0,0.0
2,16,1.0,1.0,0.0
3,20,19.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,85.0
1,12,Released,0.0
2,16,Released,1.0
3,20,Released,19.0
4,10,Executing,17.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,16,20
Status,,,,
Completed,8.0,0.0,0.0,0.0
Executing,17.0,1.0,1.0,0.0
Released,85.0,0.0,1.0,19.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,16,20
0,Completed,8.0,0.0,0.0,0.0
1,Executing,17.0,1.0,1.0,0.0
2,Released,85.0,0.0,1.0,19.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,16,20
0,Completed,8.0,0.0,0.0,0.0
1,Executing,17.0,1.0,1.0,0.0
2,Released,85.0,0.0,1.0,19.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()